In [ ]:
# bộ dữ liệu tổng , cả node và edge chưa lọc 
import torch
from ogb.nodeproppred import NodePropPredDataset
import pandas as pd

# 1. Tải và chuẩn bị dữ liệu
dataset = NodePropPredDataset(name='ogbn-arxiv')
graph, labels = dataset[0]  # Dữ liệu đồ thị và nhãn của các node

# Chuyển đổi dữ liệu về dạng Tensor
node_features = torch.tensor(graph['node_feat'], dtype=torch.float)  # Đặc trưng node
edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)  # Cạnh
labels = torch.tensor(labels, dtype=torch.long).squeeze()  # Nhãn node

# 2. Lưu tập đỉnh ra CSV
def save_nodes_to_csv(node_features, labels, filename):
    node_ids = torch.arange(node_features.shape[0])  # ID của các node
    node_data = pd.DataFrame({
        'node_id': node_ids.numpy(),
        'label': labels.numpy()
    })
    
    # Lấy đặc trưng của các node
    node_features_df = pd.DataFrame(node_features.numpy())
    
    # Gộp thông tin node và đặc trưng
    node_data = pd.concat([node_data, node_features_df], axis=1)
    
    # Lưu vào file CSV
    node_data.to_csv(filename, index=False)

# Lưu tập đỉnh
save_nodes_to_csv(node_features, labels, 'nodes.csv')

# 3. Lưu tập cạnh ra CSV
def save_edges_to_csv(edge_index, filename):
    edge_data = pd.DataFrame({
        'source': edge_index[0].numpy(),
        'target': edge_index[1].numpy()
    })
    
    # Lưu vào file CSV
    edge_data.to_csv(filename, index=False)

# Lưu tập cạnh
save_edges_to_csv(edge_index, 'edges.csv')

print("Nodes and edges saved to CSV!")


: 

In [ ]:
### LỌC CÁC NODE ra CSV trước

# Tổng các node = ở 3 file train, test, val = số node tổng

import torch
from ogb.nodeproppred import NodePropPredDataset
import pandas as pd

# Lọc các node ra CSV

# 1. Tải và chuẩn bị dữ liệu
dataset = NodePropPredDataset(name='ogbn-arxiv')
split_idx = dataset.get_idx_split()  # Indices của train, val, test
graph, labels = dataset[0]  # Dữ liệu đồ thị và nhãn của các node

# Chuyển đổi dữ liệu về dạng Tensor
node_features = torch.tensor(graph['node_feat'], dtype=torch.float)  # Đặc trưng node
labels = torch.tensor(labels, dtype=torch.long).squeeze()  # Nhãn node

train_idx = torch.tensor(split_idx['train'])
val_idx = torch.tensor(split_idx['valid'])
test_idx = torch.tensor(split_idx['test'])

# 2. Lưu dữ liệu ra CSV (chỉ lưu thông tin node)
def save_to_csv(node_idx, node_features, labels, prefix):
    # Node data
    node_data = pd.DataFrame({
        'node_id': node_idx.numpy(),
        'label': labels[node_idx].numpy()
    })
    node_features_df = pd.DataFrame(node_features[node_idx].numpy())
    node_data = pd.concat([node_data, node_features_df], axis=1)
    node_data.to_csv(f'{prefix}_nodes.csv', index=False)

# Lưu train, val, test
save_to_csv(train_idx, node_features, labels, 'train')
save_to_csv(val_idx, node_features, labels, 'val')
save_to_csv(test_idx, node_features, labels, 'test')

print("Data saved to CSV!")


In [35]:
import pandas as pd

# Đọc dữ liệu
edges = pd.read_csv("edges.csv")
train_nodes = pd.read_csv("train_nodes.csv")
test_nodes = pd.read_csv("test_nodes.csv")
val_nodes = pd.read_csv("val_nodes.csv")

# Tạo tập hợp các node_id từ mỗi tập
train_node_ids = set(train_nodes['node_id'])
test_node_ids = set(test_nodes['node_id'])
val_node_ids = set(val_nodes['node_id'])

# Lọc cạnh cho train_edges
train_edges = edges[
    (edges['source'].isin(train_node_ids)) & 
    (edges['target'].isin(train_node_ids))
]
train_edges.to_csv("train_edges.csv", index=False)
print(f"Train edges: {len(train_edges)} cạnh đã được lưu.")

# Lọc cạnh cho test_edges
test_edges = edges[
    (edges['source'].isin(test_node_ids)) & 
    (edges['target'].isin(test_node_ids))
]
test_edges.to_csv("test_edges.csv", index=False)
print(f"Test edges: {len(test_edges)} cạnh đã được lưu.")

# Lọc cạnh cho val_edges
val_edges = edges[
    (edges['source'].isin(val_node_ids)) & 
    (edges['target'].isin(val_node_ids))
]
val_edges.to_csv("val_edges.csv", index=False)
print(f"Validation edges: {len(val_edges)} cạnh đã được lưu.")

print("Lọc cạnh hoàn tất.")


Train edges: 374839 cạnh đã được lưu.
Test edges: 60403 cạnh đã được lưu.
Validation edges: 29119 cạnh đã được lưu.
Lọc cạnh hoàn tất.


In [37]:
import pandas as pd

# do các đỉnh nó ko liên tục nên cần ánh xạ đại phù hợp với đầu vào cho mô hình 

# Đọc dữ liệu
train_nodes = pd.read_csv("train_nodes.csv")  # Các node trong tập train
train_edges = pd.read_csv("train_edges.csv")  # Các edge trong tập train

# Tạo ánh xạ node_id cho tập train
train_node_ids = train_nodes['node_id'].unique()
train_node_mapping = {node_id: new_id for new_id, node_id in enumerate(train_node_ids)}

# Ánh xạ lại node_id trong train_nodes
train_nodes['node_id'] = train_nodes['node_id'].map(train_node_mapping)

# Ánh xạ lại source và target trong train_edges
train_edges['source'] = train_edges['source'].map(train_node_mapping)
train_edges['target'] = train_edges['target'].map(train_node_mapping)

# Lưu lại kết quả
train_nodes.to_csv("train_nodes.csv", index=False)
train_edges.to_csv("train_edges.csv", index=False)

print("Hoàn tất ánh xạ lại node_id cho tập train.")


Hoàn tất ánh xạ lại node_id cho tập train.


In [39]:
# Đọc dữ liệu
test_nodes = pd.read_csv("test_nodes.csv")  # Các node trong tập test
test_edges = pd.read_csv("test_edges.csv")  # Các edge trong tập test

# Tạo ánh xạ node_id cho tập test
test_node_ids = test_nodes['node_id'].unique()
test_node_mapping = {node_id: new_id for new_id, node_id in enumerate(test_node_ids)}

# Ánh xạ lại node_id trong test_nodes
test_nodes['node_id'] = test_nodes['node_id'].map(test_node_mapping)

# Ánh xạ lại source và target trong test_edges
test_edges['source'] = test_edges['source'].map(test_node_mapping)
test_edges['target'] = test_edges['target'].map(test_node_mapping)

# Lưu lại kết quả
test_nodes.to_csv("test_nodes.csv", index=False)
test_edges.to_csv("test_edges.csv", index=False)

print("Hoàn tất ánh xạ lại node_id cho tập test.")


Hoàn tất ánh xạ lại node_id cho tập test.


In [40]:
# Đọc dữ liệu
val_nodes = pd.read_csv("val_nodes.csv")  # Các node trong tập val
val_edges = pd.read_csv("val_edges.csv")  # Các edge trong tập val

# Tạo ánh xạ node_id cho tập val
val_node_ids = val_nodes['node_id'].unique()
val_node_mapping = {node_id: new_id for new_id, node_id in enumerate(val_node_ids)}

# Ánh xạ lại node_id trong val_nodes
val_nodes['node_id'] = val_nodes['node_id'].map(val_node_mapping)

# Ánh xạ lại source và target trong val_edges
val_edges['source'] = val_edges['source'].map(val_node_mapping)
val_edges['target'] = val_edges['target'].map(val_node_mapping)

# Lưu lại kết quả
val_nodes.to_csv("val_nodes.csv", index=False)
val_edges.to_csv("val_edges.csv", index=False)

print("Hoàn tất ánh xạ lại node_id cho tập val.")


Hoàn tất ánh xạ lại node_id cho tập val.


In [49]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd

# 1. Định nghĩa mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# 2. Load dữ liệu từ file CSV
def load_data():
    nodes = pd.read_csv("train_nodes.csv")
    edges = pd.read_csv("train_edges.csv")

    x = torch.tensor(nodes.iloc[:, 2:].values, dtype=torch.float)
    y = torch.tensor(nodes['label'].values, dtype=torch.long)
    edge_index = torch.tensor(edges[['source', 'target']].values.T, dtype=torch.long)

    num_nodes = x.size(0)
    train_mask = torch.ones(num_nodes, dtype=torch.bool)

    data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask)
    return data

# 3. Huấn luyện mô hình
def train(model, data, optimizer, num_epochs=100):
    model.train()
    for epoch in range(1, num_epochs + 1):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        train_acc = test(model, data, data.train_mask)
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item()}, Training Accuracy: {train_acc:.4f}")

# 4. Kiểm tra độ chính xác
def test(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
    return acc

# 5. Lưu trọng số mô hình
def save_model(model, path="gcn_model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model weights saved to {path}")

# 6. Chạy huấn luyện
if __name__ == "__main__":
    data = load_data()
    in_channels = data.x.shape[1]
    hidden_channels = 64
    out_channels = len(torch.unique(data.y))

    model = GCN(in_channels, hidden_channels, out_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    train(model, data, optimizer, num_epochs=200)
    save_model(model)


Epoch 1/200, Loss: 3.8153817653656006, Training Accuracy: 0.0901
Epoch 2/200, Loss: 3.431565523147583, Training Accuracy: 0.2123
Epoch 3/200, Loss: 3.2221832275390625, Training Accuracy: 0.2130
Epoch 4/200, Loss: 3.10653018951416, Training Accuracy: 0.2572
Epoch 5/200, Loss: 3.0621628761291504, Training Accuracy: 0.2611
Epoch 6/200, Loss: 3.025890827178955, Training Accuracy: 0.2612
Epoch 7/200, Loss: 2.9734301567077637, Training Accuracy: 0.2720
Epoch 8/200, Loss: 2.9261839389801025, Training Accuracy: 0.3004
Epoch 9/200, Loss: 2.8895812034606934, Training Accuracy: 0.3056
Epoch 10/200, Loss: 2.8428330421447754, Training Accuracy: 0.3027
Epoch 11/200, Loss: 2.7882943153381348, Training Accuracy: 0.2982
Epoch 12/200, Loss: 2.7357568740844727, Training Accuracy: 0.3030
Epoch 13/200, Loss: 2.6874444484710693, Training Accuracy: 0.3186
Epoch 14/200, Loss: 2.6435256004333496, Training Accuracy: 0.3447
Epoch 15/200, Loss: 2.6035408973693848, Training Accuracy: 0.3645
Epoch 16/200, Loss: 2.5

In [58]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd

# fine-tune tiếp với tập test

# 1. Định nghĩa mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# 2. Load dữ liệu từ file CSV
def load_data(node_file, edge_file):
    nodes = pd.read_csv(node_file)
    edges = pd.read_csv(edge_file)

    x = torch.tensor(nodes.iloc[:, 2:].values, dtype=torch.float)
    y = torch.tensor(nodes['label'].values, dtype=torch.long)
    edge_index = torch.tensor(edges[['source', 'target']].values.T, dtype=torch.long)

    num_nodes = x.size(0)
    train_mask = torch.ones(num_nodes, dtype=torch.bool)

    data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask)
    return data

# 3. Load và tinh chỉnh mô hình
def load_and_finetune(model, path, data, optimizer, num_epochs=100):
    model.load_state_dict(torch.load(path))
    model.train()
    for epoch in range(1, num_epochs + 1):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        train_acc = test(model, data, data.train_mask)
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item()}, Training Accuracy: {train_acc:.4f}")

# 4. Kiểm tra độ chính xác
def test(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
    return acc

# 5. Lưu trọng số mô hình
def save_model(model, path="gcn_model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model weights saved to {path}")

# 6. Chạy huấn luyện/tinh chỉnh
if __name__ == "__main__":
    # Load dữ liệu mới
    data = load_data("test_nodes.csv", "test_edges.csv")
    in_channels = data.x.shape[1]
    hidden_channels = 64
    out_channels = len(torch.unique(data.y))

    # Khởi tạo mô hình và optimizer
    model = GCN(in_channels, hidden_channels, out_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # Load và tinh chỉnh mô hình
    load_and_finetune(model, "gcn_model.pth", data, optimizer, num_epochs=100)

    # Lưu trọng số (tùy chọn)
    save_model(model)


C:\Users\HOANG\AppData\Local\Temp\ipykernel_16252\3194151381.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


Epoch 1/100, Loss: 1.3495415449142456, Training Accuracy: 0.5335
Epoch 2/100, Loss: 1.5639960765838623, Training Accuracy: 0.5759
Epoch 3/100, Loss: 1.4515894651412964, Training Accuracy: 0.5731
Epoch 4/100, Loss: 1.4540361166000366, Training Accuracy: 0.5679
Epoch 5/100, Loss: 1.453067660331726, Training Accuracy: 0.5823
Epoch 6/100, Loss: 1.4111069440841675, Training Accuracy: 0.5792
Epoch 7/100, Loss: 1.3976287841796875, Training Accuracy: 0.5832
Epoch 8/100, Loss: 1.3990449905395508, Training Accuracy: 0.5822
Epoch 9/100, Loss: 1.4100922346115112, Training Accuracy: 0.5881
Epoch 10/100, Loss: 1.3944059610366821, Training Accuracy: 0.5962
Epoch 11/100, Loss: 1.3695452213287354, Training Accuracy: 0.5967
Epoch 12/100, Loss: 1.365407109260559, Training Accuracy: 0.5915
Epoch 13/100, Loss: 1.3762049674987793, Training Accuracy: 0.5914
Epoch 14/100, Loss: 1.3819260597229004, Training Accuracy: 0.5948
Epoch 15/100, Loss: 1.3724029064178467, Training Accuracy: 0.5982
Epoch 16/100, Loss: 1

In [61]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd

# 1. Định nghĩa mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x  # Trả về trực tiếp các đặc trưng ẩn

# 2. Load dữ liệu từ file CSV
def load_data(node_file, edge_file):
    nodes = pd.read_csv(node_file)
    edges = pd.read_csv(edge_file)

    x = torch.tensor(nodes.iloc[:, 2:].values, dtype=torch.float)
    y = torch.tensor(nodes['label'].values, dtype=torch.long)
    edge_index = torch.tensor(edges[['source', 'target']].values.T, dtype=torch.long)

    num_nodes = x.size(0)
    train_mask = torch.ones(num_nodes, dtype=torch.bool)

    # Add node_id to Data
    data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, node_id=nodes['node_id'].values)
    return data

# 3. Load và tính toán embedding vectors
def compute_embeddings(model, path, data, output_file):
    model.load_state_dict(torch.load(path))
    model.eval()
    with torch.no_grad():
        embeddings = model(data.x, data.edge_index)
        node_ids = data['node_id']  # Lấy node_id từ tập dữ liệu

        # Tạo DataFrame và lưu xuống CSV
        embeddings_df = pd.DataFrame(embeddings.numpy(), columns=[f"{i}" for i in range(embeddings.size(1))])
        embeddings_df['node_id'] = node_ids
        embeddings_df.to_csv(output_file, index=False)
        print(f"Embedding vectors with node_id saved to {output_file}")

# 4. Chạy
if __name__ == "__main__":
    data = load_data("val_nodes.csv", "val_edges.csv")
    in_channels = data.x.shape[1]
    hidden_channels = 64
    out_channels = len(torch.unique(data.y))

    model = GCN(in_channels, hidden_channels, out_channels)

    # Tính toán các embedding vectors và lưu ra CSV
    compute_embeddings(model, "gcn_model.pth", data, "embedding_val.csv")


C:\Users\HOANG\AppData\Local\Temp\ipykernel_16252\520124768.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


Embedding vectors with node_id saved to embedding_val.csv


In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd

# 1. Định nghĩa mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x  # Trả về trực tiếp các đặc trưng ẩn

# 2. Load dữ liệu từ file CSV
def load_data(node_file, edge_file):
    nodes = pd.read_csv(node_file)
    edges = pd.read_csv(edge_file)

    x = torch.tensor(nodes.iloc[:, 2:].values, dtype=torch.float)
    y = torch.tensor(nodes['label'].values, dtype=torch.long)
    edge_index = torch.tensor(edges[['source', 'target']].values.T, dtype=torch.long)

    num_nodes = x.size(0)
    train_mask = torch.ones(num_nodes, dtype=torch.bool)

    # Add node_id to Data
    data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, node_id=nodes['node_id'].values)
    return data

# 3. Đánh giá mô hình trên tập validation
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = pred[data.train_mask].eq(data.y[data.train_mask]).sum().item()
        accuracy = correct / data.train_mask.sum().item()
        print(f"Validation Accuracy: {accuracy:.4f}")

# 4. Chạy
if __name__ == "__main__":
    data = load_data("val_nodes.csv", "val_edges.csv")
    in_channels = data.x.shape[1]
    hidden_channels = 64
    out_channels = len(torch.unique(data.y))

    model = GCN(in_channels, hidden_channels, out_channels)

    # Load model weights
    model.load_state_dict(torch.load("gcn_model.pth"))

    # Đánh giá mô hình trên tập validation
    evaluate(model, data)


Validation Accuracy: 0.5683


C:\Users\HOANG\AppData\Local\Temp\ipykernel_3324\707412869.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("gcn_model.pth"))
